# Fade in and out effects

In [3]:
%use jupyter-wave
%use lets-plot

## Fade functions implementation

In [24]:
val fadeInMix = input { (sampleIndex, sampleRate) ->
    // parameter `kick in time` in seconds
    val kickInTime = 0.0
    // parameter `duration` in seconds
    val duration = 0.05
    // calculate the kick in time in samples based on current sample rate
    val fadeInMinAt = sampleRate * kickInTime
    // calculate duration in samples based on current sample rate
    val durationInSample = sampleRate * duration
    // calculate the position in interval making sure it is inside the interval [0.0, 1.0]
    min(
        max(
            (sampleIndex - fadeInMinAt) / durationInSample,  
            0.0
        ),
        1.0
    )
}

fadeInMix.trim(70).plot(sampleRate = 11025.0f)

In [25]:
val fadeOutMix = input { (sampleIndex, sampleRate) ->
    // parameter `kick in time` in seconds
    val kickInTime = 0.02
    // parameter `duration` in seconds
    val duration = 0.05
    // calculate the kick in time in samples based on current sample rate
    val fadeOutMaxAt = sampleRate * kickInTime
    // calculate duration in samples based on current sample rate
    val durationInSample = sampleRate * duration
    // calculate the position in interval making sure it is inside the interval [0.0, 1.0]
    min(
        max(
            1.0 - (sampleIndex - fadeOutMaxAt) / durationInSample,
            0.0
        ),
        1.0
    )
}

fadeOutMix.trim(70).plot(sampleRate = 11025.0f)

## Mix it in

In [26]:
val s = 440.sine()

s.trim(70).plot(sampleRate = 11025.0f)

In [27]:
var r1 = s * fadeInMix

r1.trim(70).plot(sampleRate = 11025.0f)

In [28]:
var r2 = s * fadeOutMix

r2.trim(70).plot(sampleRate = 11025.0f)

In [29]:
var r3 = s * fadeInMix * fadeOutMix

r3.trim(70).plot(sampleRate = 11025.0f)

## Different fade in-out functions

In [30]:
val fadeInLogMix = input { (sampleIndex, sampleRate) ->
    // parameter `kick in time` in seconds
    val kickInTime = 0.0
    // parameter `duration` in seconds
    val duration = 0.05
    // parameter `base` of the logarithmic function
    val base = 10.0
    // calculate the kick in time in samples based on current sample rate
    val fadeInMinAt = sampleRate * kickInTime
    // calculate duration in samples based on current sample rate
    val durationInSamples = sampleRate * duration
    // calculate the argument of the algorithmic function, it should be >= 0.0
    val x = max(sampleIndex - fadeInMinAt, 0.0) / durationInSamples
    // calculate the position in interval using logarithm function
    // and make sure it is inside the interval [0.0, 1.0]
    min(
      log(x * (base - 1.0) + 1.0, base),
      1.0
    )
}

fadeInLogMix.trim(70).plot(sampleRate = 11025.0f)

In [31]:
(s * fadeInLogMix).trim(70).plot(sampleRate = 11025.0f)

In [32]:
val fadeOutLogMix = input { (sampleIndex, sampleRate) ->
    // parameter `kick in time` in seconds
    val kickInTime = 0.02
    // parameter `duration` in seconds
    val duration = 0.05
    // parameter `base` of the logarithmic function
    val base = 10.0
    // calculate the kick in time in samples based on current sample rate
    val fadeOutMaxAt = sampleRate * kickInTime
    // calculate duration in samples based on current sample rate
    val durationInSamples = sampleRate * duration
    // calculate the argument of the algorithmic function, it should be >= 0.0
    val x = max(sampleIndex - fadeOutMaxAt, 0.0) / durationInSamples
    // calculate the position in interval using logarithm function
    // and make sure it is inside the interval [0.0, 1.0]
    max(
        min(
          log((1.0 - x) * (base - 1.0) + 1.0, base),
          1.0
        ),
        0.0
    )
}

fadeOutLogMix.trim(70).plot(sampleRate = 11025.0f)

In [33]:
(s * fadeOutLogMix).trim(70).plot(sampleRate = 11025.0f)

### Difference

In [72]:
import kotlin.math.*

class Fade(initParams: FnInitParameters): Fn<Pair<Long, Float>, Sample?>(initParams) {
    
    constructor(
        kickInTime: Double, // in seconds
        duration: Double, // in seconds
        fn: (Double) -> Double // mapping interval [0.0, 1.0] to an actual multiplier
    ) : this(
        FnInitParameters()
            .add("kickInTime", kickInTime)
            .add("duration", duration)
            .add("fn", Fn.wrap(fn))
    )
    
    private val kickInTime = initParams.double("kickInTime")
    private val duration = initParams.double("duration")
    private val fn: Fn<Double, Double> = initParams.fn("fn")

    override fun apply(argument: Pair<Long, Float>): Sample? {
        val (sampleIndex, sampleRate) = argument
        val fadeMinAt = sampleRate * kickInTime
        val durationInSamples = sampleRate * duration
        val x = max(sampleIndex - fadeMinAt, 0.0) / durationInSamples
        // limit with interval [0.0, 1.0] and return as sample
        return sampleOf(max(min(fn.apply(x), 1.0), 0.0)) 
    }
}

object logFade {
    val base = 10.0
}


val fadeInMix2 = input(Fade(.0, .03) { it } )
val fadeOutMix2 = input(Fade(.04, .03) { 1 - it } )
val fadeInLogMix2 = input(Fade(.0, .03) { log(it * (logFade.base - 1) + 1.0, logFade.base)} )
val fadeOutLogMix2 = input(Fade(.04, .03) { log((1 - it) * (logFade.base - 1) + 1.0, logFade.base)} )

val linear: List<Sample> = (s * fadeInMix2 * fadeOutMix2)
    .trim(70).asSequence(sampleRate = 11025.0f).toList()
val log: List<Sample> = (s * fadeInLogMix2 * fadeOutLogMix2)
    .trim(70).asSequence(sampleRate = 11025.0f).toList()

val dataFrame = mapOf(
    "time, ms" to linear.indices.asSequence().map { it / 11025.0f * 1000.0 }.toList(),
    "linear" to linear,
    "log" to log
)

lets_plot()  +
            geom_line(dataFrame, color = "blue") { x = "time, ms"; y = "linear" } + 
            geom_line(dataFrame, color = "red") { x = "time, ms"; y = "log" }